# Het effect van internationale studenten op de scores van het land

Groep 13 - Vedran Marinović, Frank Tamer, Alfonso Warmerdam, Boudewijn Welkzijn

In [14]:
import common
from common import *
from importlib import reload
reload(common)

/Users/gingerdwarf/datavisualisatie/common.py:22: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



<module 'common' from '/Users/gingerdwarf/datavisualisatie/common.py'>

## Inleiding

Het aantal buitenlandse studenten groeit over de hele wereld. In Nederland is het aantal buitenlandse studenten in collegejaar 2018-2019 toegenomen tot 55.000. De groei is ook te zien in andere landen, bijvoorbeeld in de Verenigde Staten. Naar verwachting zal het aantal internationale studenten in de VS in 2025 tegen de acht miljoen aan zitten, groeiend van vijf miljoen in 2014. Deze data story zal zich richten op het effect van deze groei op de prestaties van de universiteiten, geleid door de stelling: “Een hoger percentage internationale studenten leidt tot een hogere plaats op de wereldranglijst en tot een betere internationale reputatie”.

In [15]:
groupdivided_barchart()

Allereerst wordt de structuur van de data story uitgelegd. De story is opgesplitst in twee delen: in de eerste helft wordt gekeken naar de verschillen tussen de top 200 universiteiten, verdeeld in vier groepen; in de tweede helft wordt gekeken naar verschillen op internationaal niveau. Dit wordt gedaan om een conclusie te kunnen trekken met betrekking tot verschillen in de internationale en/of totale score. Wanneer de top 200 in vier groepen wordt gesplitst, is te zien dat het gemiddelde percentage internationale studenten tussen groepen weinig verschilt, zie bovenstaande plot.

In [16]:
boxplot_intstudents()

Hierboven is de spreiding van het percentage internationale studenten per groep van 50 universiteiten te zien. Bij de groep op rang 1-50 is te zien dat de spreiding en het gemiddelde het grootste zijn. Tussen de andere drie groepen zijn de verschillen aanzienlijk kleiner. De eerste groep heeft dus meer universiteiten met een hoger percentage internationale studenten, maar ook veel met een lager percentage. Dit weegt op tegen elkaar, wat het resultaat uit de vorige bar chart geeft.

In [17]:
scatter_intscore_intstudent()

In bovenstaande visualisatie is te zien dat het percentage internationale studenten invloed heeft op de internationale score van de universiteiten. De plot gaat diagonaal naar boven, dit laat zien dat universiteiten met een hoge internationale score ook een hoog percentage internationale studenten hebben. Rechtsboven zijn universiteiten te vinden uit verschillende groepen, dit staat dus los van de ranglijst. Wel valt op dat voornamelijk universiteiten op plekken 1-50 dominant zijn in dit gebied van de grafiek, wat aangeeft dat deze, hoger scorende, universiteiten een hoger percentage internationale studenten hebben.

In [18]:
barchart_country_intstudents()

De grafiek hierboven laat alle landen zien uit de dataset in de jaren 2011 tot 2016. Deze landen worden oplopend weergegeven op basis van het gemiddelde percentage internationale studenten over alle universiteiten binnen dat land. Deze visualisatie is van belang om in de grafieken hieronder te kijken bij landen met een hoger gemiddeld percentage internationale studenten of deze ook een hogere gemiddelde totale score hebben.

In [19]:
worldmap_intstudent('total_score', 'Gemiddelde totale score per land', 'Totale score', 100, False)

In de bovenstaande en onderstaande grafieken is te zien, indien ze worden bekeken samen de bar chart over alle landen, dat landen met een hoger percentage internationale studenten niet altijd hoger scoren. In sommige gevallen scoren deze landen zelfs lager.

In [20]:
dotplot_totalscore()

## Conclusie

De stelling die deze data story leidde was: “Een hoger percentage internationale studenten leidt tot een hogere plaats op de wereldranglijst en tot een betere internationale reputatie”. Deze stelling kan deels als bewezen worden beschouwd en deels als weerlegd. Uit de laatste grafieken is namelijk op te maken dat landen met een hoger gemiddeld percentage internationale studenten niet per se hoger scoren. Het is echter ook te zien dat landen met een hoger gemiddeld percentage internationale studenten een betere internationale reputatie hebben, een hogere internationale score.